In [1]:
import numpy as np
import torch
import random
from datetime import datetime

seed = 1234
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
# experimental configuration
data_dir = '../data/cifar10'
batch_size_train = 512
batch_size_test = 512
learning_rate = 0.01
momentum = 0.0 # momentum used in SGD optimizer, defaults to be 0.0
epochs = 200
stop_criterion = 0.01
log_interval = 1
num_workers = 4
gpu_id = 0
device = torch.device(f'cuda:{gpu_id}' if torch.cuda.is_available() else 'cpu')

In [3]:
# define the dataloader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

NRM  = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
TT   = transforms.ToTensor()
transform = transforms.Compose([TT, NRM])
train_set = datasets.CIFAR10(root=data_dir, train=True, transform=transform, download=True)
test_set = datasets.CIFAR10(root=data_dir, train=False, transform=transform, download=True)
train_loader = DataLoader(train_set, batch_size=batch_size_train, shuffle=True, num_workers=num_workers, pin_memory=True, drop_last=True)
test_loader = DataLoader(test_set, batch_size=batch_size_test, shuffle=False, num_workers=num_workers, pin_memory=True, drop_last=True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# define the model
import torch.optim as optim
import torch.nn as nn
from resnet import resnet18

net = resnet18(num_classes=10).to(device)
optimizer = optim.SGD(params=net.parameters(), lr=learning_rate, momentum=momentum)
loss_function = nn.CrossEntropyLoss(reduction='none')

In [5]:
# evaluation function
def model_test(net, eval_loader, loss_function, device):
    net.eval()
    test_loss = 0.0
    correct = 0.0
    loss_vec = []
    with torch.no_grad():
        for data, target in eval_loader:
            data, target = data.to(device), target.to(device)
            output = net(data)
            loss = loss_function(output, target)
            loss_vec.append(loss.data)
            test_loss += loss.sum()
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum()
    loss_vec = torch.hstack(loss_vec)
    if loss_vec.is_cuda:
        loss_vec = loss_vec.cpu()
    return correct.item() / len(eval_loader.dataset), test_loss.item() / len(eval_loader.dataset), loss_vec

In [6]:
# train a model
loss_vec_trace_train = []
trace_gap = []
for epoch in range(epochs):
    if epoch % log_interval == 0:
        train_acc, train_loss, train_loss_vec = model_test(net, train_loader, loss_function, device)
        test_acc, test_loss, _ = model_test(net, test_loader, loss_function, device)
        loss_vec_trace_train.append(train_loss_vec)
        trace_gap.append(train_acc-test_acc)
        print(f'Epoch {epoch}, train_acc/train_loss: {train_acc:.4f}/{train_loss:.4f}, test_acc/test_loss: {test_acc:.4f}/{test_loss:.4f}')
    if train_loss <= stop_criterion:
        break
    net.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        output = net(data)
        loss = loss_function(output, target).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
loss_vec_trace_train = torch.vstack(loss_vec_trace_train)

Epoch 0, train_acc/train_loss: 0.0945/2.4239, test_acc/test_loss: 0.0929/2.3717
Epoch 1, train_acc/train_loss: 0.3543/1.7930, test_acc/test_loss: 0.3426/1.7679
Epoch 2, train_acc/train_loss: 0.4405/1.5550, test_acc/test_loss: 0.4211/1.5485
Epoch 3, train_acc/train_loss: 0.4907/1.4116, test_acc/test_loss: 0.4586/1.4278
Epoch 4, train_acc/train_loss: 0.5273/1.3061, test_acc/test_loss: 0.4799/1.3464
Epoch 5, train_acc/train_loss: 0.5655/1.2107, test_acc/test_loss: 0.5098/1.2817
Epoch 6, train_acc/train_loss: 0.5920/1.1395, test_acc/test_loss: 0.5212/1.2422
Epoch 7, train_acc/train_loss: 0.6087/1.0881, test_acc/test_loss: 0.5345/1.2129
Epoch 8, train_acc/train_loss: 0.6381/1.0117, test_acc/test_loss: 0.5487/1.1772
Epoch 9, train_acc/train_loss: 0.6679/0.9432, test_acc/test_loss: 0.5608/1.1415
Epoch 10, train_acc/train_loss: 0.6942/0.8767, test_acc/test_loss: 0.5704/1.1228
Epoch 11, train_acc/train_loss: 0.7314/0.7910, test_acc/test_loss: 0.5841/1.0908
Epoch 12, train_acc/train_loss: 0.7327

In [7]:
# estimate the diameter of the evaluated losses
import cyminiball as miniball

_, r = miniball.compute(loss_vec_trace_train)
diameter = 2.0 * np.sqrt(r)

In [8]:
# estimate the gradients by backpropagating a number of mini-batches
from utils import cycle_loader, get_grads, get_param

num_components = 50000 # number of parameters used to estimate Hurst parameter 
len_of_sequence = 1000 # number of mini-batches to generate a stochastic sequence
cycle_train_loader = cycle_loader(train_loader)
_, tot_param = get_param(net)
if tot_param < num_components:
    num_components = tot_param
ids = torch.randperm(tot_param)[:num_components]
grads = []
for j, (data, target) in enumerate(cycle_train_loader):
    if j == len_of_sequence:
        break
    data, target = data.to(device), target.to(device)
    output = net(data)
    loss = loss_function(output, target).mean()

    optimizer.zero_grad()
    loss.backward()
    grad = get_grads(net)
    if grad.is_cuda:
        grad = grad.cpu()
    grads.append(grad[ids])
grads = torch.vstack(grads)
if grads.is_cuda:
    grads = grads.cpu()
grads = grads.double().numpy()
grads = grads[:, ~np.isnan(grads).any(axis=0)] # delete unvalid elements in case of nan

In [9]:
# estimate Hurst parameter
import os
from datetime import datetime
import multiprocessing as mp
from hurst import compute_Hc

start_t = datetime.now()

# here we exploit a parallel trick to accelerate the process
global get_hurst
def get_hurst(j):
    h = compute_Hc(grads[:, j], kind='random_walk')
    return h
pool = mp.Pool(os.cpu_count())
res = [pool.apply_async(get_hurst, args=(j,)) for j in range(grads.shape[1])]
res = [p.get() for p in res]
pool.close()
pool.join()

end_t = datetime.now()
elapsed_sec = (end_t - start_t).total_seconds()

# remove the outliers
res = np.array(res)
res = res[res>0.0]
res = res[res<1.0]

hurst_index = np.median(res)

print("Hurst parameter: {:.4f}, time elapsed: {:.2f} seconds".format(hurst_index, elapsed_sec))

Hurst parameter: 0.1597, time elapsed: 48.65 seconds


In [14]:
theoretical_bound =  24 * diameter / 50000 * np.sqrt(np.log(4.0) / (hurst_index + 1.0e-10))
empirical_bound = np.max(np.abs(trace_gap))
print(f'Empirical bound: {empirical_bound:.4f}, theoretical bound: {theoretical_bound:.4f}')

Empirical bound: 0.3924, theoretical bound: 0.7825
